Кобзарь О.С. Хабибуллин Р.А. 29.07.2019

# Обработка данных со станции управления при помощи библиотеки Pandas

Большое количество данных и отсутствие быстродействия Excel вынуждает искать другие способы обработки и просмотра промысловых данных. Решением может быть удобная и популярная библиотека Pandas. В данном примере тестовые исходные данные со СУ преобразуются в удобное представление для оптимальной работы: построения графиков, ресемплирования, выбора интервалов значений, последующего использования для расчетов. Все методы Excel последовательно обернуты в класс `Editor_initial_cs_data`, находящиеся в модуле `initial_data_editor` для обработки именно данного представления данных. Для обработки других представлений (что неизбежно, форматов множество) данный класс может служить примером.

Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')
import uniflocpy.uTools.data_editor as editor
import pandas as pd

Создание экземпляра класса, с помощью которого осуществляется работа с данными

In [2]:
this_editor = editor.Editor_initial_cs_data()

Загрузка данных

In [3]:
this_editor.load_csv("data/input_test_contol_station_data.csv")

Если произошла ошибка, необходимо загрузить вручную строкой ниже, предварительно раскомментировав

In [4]:
#this_editor.init_pandas_df = pd.read_csv("test_contol_station_data.csv", sep=';', header=None)

In [5]:
this_editor.init_pandas_df.head()

,0,1,2,3
0,PP.W.03,Тестовая скважина №1. Давление на приеме насос...,2019-02-28 16:10:48.8980000,2.99985
1,UT.W.06,Тестовая скважина №1. Напряжение на выходе ТМПН,2019-02-28 16:10:48.8980000,1550.00000
2,Y3.W.06,Тестовая скважина №1. Входное напряжение СА,2019-02-28 16:10:48.8980000,397.00000
3,ZD.W.06,Тестовая скважина №1. Загрузка двигателя,2019-02-28 16:10:48.8980000,51.00000
4,AS.W.06,Тестовая скважина №1. Активная мощность,2019-02-28 16:16:40.8980000,84.00000


Как видно из представления, DataFrame содержит лишние столбцы, неотсортированные значения, которыми неудобно пользоваться

Извлечем список определяющих уникальных параметров из исходных данных

In [6]:
parametrs_list = this_editor.get_parametrs_list()
print(parametrs_list)

['Тестовая скважина №1. Активная мощность'
 'Тестовая скважина №1. Вибрация насоса по оси X'
 'Тестовая скважина №1. Вибрация насоса по оси Y'
 'Тестовая скважина №1. Входное напряжение АВ'
 'Тестовая скважина №1. Входное напряжение ВС'
 'Тестовая скважина №1. Входное напряжение СА'
 'Тестовая скважина №1. Давление на приеме насоса (пласт. жидкость)'
 'Тестовая скважина №1. Дисбаланс напряжений'
 'Тестовая скважина №1. Дисбаланс тока полуходовой'
 'Тестовая скважина №1. Дисбаланс токов'
 'Тестовая скважина №1. Загрузка двигателя'
 'Тестовая скважина №1. Коэффициент мощности'
 'Тестовая скважина №1. Нагрузка на шток максимальная'
 'Тестовая скважина №1. Напряжение в звене постоянного тока'
 'Тестовая скважина №1. Напряжение на выходе ПЧ'
 'Тестовая скважина №1. Напряжение на выходе ТМПН'
 'Тестовая скважина №1. Общее время работы насоса'
 'Тестовая скважина №1. Полная мощность'
 'Тестовая скважина №1. Сопротивление изоляции'
 'Тестовая скважина №1. Температура обмоток двигателя'
 'Тесто

Получим информацию по названиям параметров из исходных данных

In [7]:
this_editor.get_information()

№0 Кол-во записей: 69  Тестовая скважина №1. Активная мощность
№1 Кол-во записей: 6  Тестовая скважина №1. Вибрация насоса по оси X
№2 Кол-во записей: 6  Тестовая скважина №1. Вибрация насоса по оси Y
№3 Кол-во записей: 44  Тестовая скважина №1. Входное напряжение АВ
№4 Кол-во записей: 37  Тестовая скважина №1. Входное напряжение ВС
№5 Кол-во записей: 43  Тестовая скважина №1. Входное напряжение СА
№6 Кол-во записей: 63  Тестовая скважина №1. Давление на приеме насоса (пласт. жидкость)
№7 Кол-во записей: 13  Тестовая скважина №1. Дисбаланс напряжений
№8 Кол-во записей: 42  Тестовая скважина №1. Дисбаланс тока полуходовой
№9 Кол-во записей: 2  Тестовая скважина №1. Дисбаланс токов
№10 Кол-во записей: 70  Тестовая скважина №1. Загрузка двигателя
№11 Кол-во записей: 51  Тестовая скважина №1. Коэффициент мощности
№12 Кол-во записей: 71  Тестовая скважина №1. Нагрузка на шток максимальная
№13 Кол-во записей: 46  Тестовая скважина №1. Напряжение в звене постоянного тока
№14 Кол-во записей: 3

Произведем первончальную обработку и увидим результат 

In [8]:
str_to_delete = "Тестовая скважина №1. "
init_edited_data = this_editor.initial_editing(str_to_delete)
init_edited_data.head()

,1,3
2,,
2019-02-28 16:10:48.898,Давление на приеме насоса (пласт. жидкость),2.99985
2019-02-28 16:10:48.898,Напряжение на выходе ТМПН,1550.00000
2019-02-28 16:10:48.898,Входное напряжение СА,397.00000
2019-02-28 16:10:48.898,Загрузка двигателя,51.00000
2019-02-28 16:16:40.898,Активная мощность,84.00000


Создадим список параметров для первоначально обработанных данных

In [9]:
edited_parametrs_list = this_editor.get_parametrs_list(init_edited_data)

Для демонстрации извлечем данные для одного параметра. Далее с помощью метода create_edited_df из подготовленного DataFrame будут извлекаться все параметры этим же способом

In [10]:
one_parametr_df = this_editor.extract_df_one_parametr_and_edit(init_edited_data, edited_parametrs_list, 0)
one_parametr_df.head()

,Активная мощность
2,
2019-02-28 16:16:40.898000,84.0
2019-02-28 16:21:04.898000,81.0
2019-02-28 16:25:42.898000,85.0
2019-02-28 16:31:39.898000,75.0
2019-02-28 16:36:18.898000,84.0


Набор данных по одному параметров при слиянии образует единую таблицу, в которой параметры разнесены по столбцам, а записи параметров отсортированы по времени

In [11]:
edited_df = this_editor.create_edited_df(init_edited_data, edited_parametrs_list)
edited_df.head()

C:\Anaconda\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Активная мощность,Вибрация насоса по оси X,Вибрация насоса по оси Y,Входное напряжение АВ,Входное напряжение ВС,Входное напряжение СА,Давление на приеме насоса (пласт. жидкость),Дисбаланс напряжений,Дисбаланс тока полуходовой,Дисбаланс токов,...,Напряжение в звене постоянного тока,Напряжение на выходе ПЧ,Напряжение на выходе ТМПН,Общее время работы насоса,Полная мощность,Сопротивление изоляции,Температура обмоток двигателя,Ток фазы А,Ток фазы В,Ток фазы С
2019-02-28 16:10:48.898000,NaN,NaN,NaN,NaN,NaN,397.0,2.99985,NaN,NaN,NaN,...,NaN,NaN,1550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-28 16:16:40.898000,84.0,NaN,NaN,NaN,NaN,NaN,2.99005,NaN,NaN,NaN,...,504.0,NaN,1544.0,NaN,116.0,NaN,NaN,43.599998,43.299999,44.799999
2019-02-28 16:21:04.898000,81.0,NaN,NaN,NaN,400.0,396.0,NaN,1.0,24.0,NaN,...,505.0,NaN,NaN,NaN,114.0,2613.0,NaN,42.500000,42.200001,43.700001
2019-02-28 16:25:42.897000,NaN,NaN,NaN,NaN,399.0,397.0,NaN,0.0,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,9999.0,NaN,NaN,NaN,NaN
2019-02-28 16:25:42.898000,85.0,NaN,NaN,NaN,NaN,NaN,3.00966,NaN,NaN,NaN,...,506.0,NaN,1550.0,NaN,117.0,NaN,NaN,43.900002,43.599998,45.099998


Запишем измененный DataFrame в csv

In [12]:
#edited_df.to_csv("simple_edited_data.csv", index = True, index_label = "Time")
csv_name_str = "data/edited_test_contol_station_data.csv"
this_editor.save_edited_df(edited_df, csv_name_str)

Для проверки загрузим его

In [13]:
loaded_edited_df = pd.read_csv(csv_name_str, index_col="Time")

In [14]:
loaded_edited_df.head()

,Активная мощность,Вибрация насоса по оси X,Вибрация насоса по оси Y,Входное напряжение АВ,Входное напряжение ВС,Входное напряжение СА,Давление на приеме насоса (пласт. жидкость),Дисбаланс напряжений,Дисбаланс тока полуходовой,Дисбаланс токов,...,Напряжение в звене постоянного тока,Напряжение на выходе ПЧ,Напряжение на выходе ТМПН,Общее время работы насоса,Полная мощность,Сопротивление изоляции,Температура обмоток двигателя,Ток фазы А,Ток фазы В,Ток фазы С
Time,,,,,,,,,,,,,,,,,,,,,
2019-02-28 16:10:48.898000,NaN,NaN,NaN,NaN,NaN,397.0,2.99985,NaN,NaN,NaN,...,NaN,NaN,1550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-28 16:16:40.898000,84.0,NaN,NaN,NaN,NaN,NaN,2.99005,NaN,NaN,NaN,...,504.0,NaN,1544.0,NaN,116.0,NaN,NaN,43.599998,43.299999,44.799999
2019-02-28 16:21:04.898000,81.0,NaN,NaN,NaN,400.0,396.0,NaN,1.0,24.0,NaN,...,505.0,NaN,NaN,NaN,114.0,2613.0,NaN,42.500000,42.200001,43.700001
2019-02-28 16:25:42.897000,NaN,NaN,NaN,NaN,399.0,397.0,NaN,0.0,23.0,NaN,...,NaN,NaN,NaN,NaN,NaN,9999.0,NaN,NaN,NaN,NaN
2019-02-28 16:25:42.898000,85.0,NaN,NaN,NaN,NaN,NaN,3.00966,NaN,NaN,NaN,...,506.0,NaN,1550.0,NaN,117.0,NaN,NaN,43.900002,43.599998,45.099998
